In [1]:
import gameoftrackers as got
import matplotlib.pyplot as plt
import numpy as np

timer_type = got.ExposureNotificationTimerTypes.EXPOSURE_NOTIFICATION_TIMER_SPEC

trackables = {}
for cluster_size in range(1, 10):
    trackables[cluster_size] = got.run_game(
        n_simulation_runs=1,
        cluster_size=cluster_size,
        triangulated=False,
        timer_type=timer_type
    )

data = np.array([
    [i+1, t]
    for i in range(len(trackables))
    for t in trackables[i+1]
])
x = data[:,0]
y = data[:,1]

plt.scatter(x=x, y=y)
plt.xlabel('cluster size')
plt.ylabel('trackable fraction')
plt.title(str(timer_type))
plt.show()

ZeroDivisionError: float division by zero